In [1]:
import glob, json, pandas as pd
from sklearn.model_selection import train_test_split

# 파일 읽기
records = []
for path in glob.glob("../data/*/*.json"):
    j = json.load(open(path, encoding="utf-8"))
    text = "\n".join(j.get("clauseArticle", []))
    label = int(j["dvAntageous"]) - 1    # 예: 1→0, 2→1
    records.append({"text": text, "label": label})

df = pd.DataFrame(records)
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
val_df, test_df  = train_test_split(test_df, test_size=0.5, stratify=test_df["label"], random_state=42)


In [ ]:
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments,
    DataCollatorWithPadding
)
import torch
from datasets import Dataset
from evaluate import load as load_metric

# 1) 토크나이저 & 모델 로드
MODEL_NAME = "monologg/koelectra-base-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2
)

# 2) Huggingface Dataset으로 변환
train_ds = Dataset.from_pandas(train_df)
val_ds   = Dataset.from_pandas(val_df)

# 3) 토크나이징 함수
def tokenize_fn(example):
    return tokenizer(
        example["text"], 
        truncation=True, 
        max_length=512
    )

train_tok = train_ds.map(tokenize_fn, batched=True)
val_tok   = val_ds.map(tokenize_fn, batched=True)

# 4) DataCollator (dynamic padding)
data_collator = DataCollatorWithPadding(tokenizer)

# 5) 평가 지표 정의 (정확도)
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return metric.compute(predictions=preds, references=labels)

# 6) Trainer 세팅
training_args = TrainingArguments(
    output_dir="./outputs",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    do_eval=True,          # old-style evaluation
    eval_steps=500,        # 원하는 스텝마다
    logging_steps=500,     # 로깅 스텝
    save_steps=500         # 체크포인트 저장 스텝
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 7) 학습 실행
trainer.train()

# 8) 테스트 평가
trainer.evaluate(Dataset.from_pandas(test_df).map(tokenize_fn, batched=True))


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 800/800 [00:00<00:00, 10991.97 examples/s]


Step,Training Loss
500,0.393700
1000,0.173200
1500,0.132300
2000,0.127400
2500,0.129500
3000,0.117700
3500,0.040300
4000,0.049600
4500,0.041000
5000,0.019300


Map: 100%|██████████| 800/800 [00:00<00:00, 7098.28 examples/s]


{'eval_loss': 0.08260364830493927,
 'eval_accuracy': 0.98875,
 'eval_runtime': 42.2248,
 'eval_samples_per_second': 18.946,
 'eval_steps_per_second': 0.592,
 'epoch': 3.0}

In [ ]:
trainer.save_model("classification")  
tokenizer.save_pretrained("classification")

('classification_model\\tokenizer_config.json',
 'classification_model\\special_tokens_map.json',
 'classification_model\\vocab.txt',
 'classification_model\\added_tokens.json',
 'classification_model\\tokenizer.json')

In [16]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# 1) 저장된 체크포인트 경로 지정
checkpoint_dir = "classification"  # Trainer 가 마지막에 저장한 폴더

# 2) 토크나이저 & 모델 불러오기
tokenizer = AutoTokenizer.from_pretrained(checkpoint_dir)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_dir)
model.eval()  # 평가 모드로 전환

# 3) 분류 함수 정의
def classify_clause(text: str) -> str:
    inputs = tokenizer(
        text,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    with torch.no_grad():
        logits = model(**inputs).logits
    pred_id = logits.argmax(dim=-1).item()
    # 레이블 맵핑 (0: 불리, 1: 유리)
    label_map = {1: "불리", 0: "유리"}
    return label_map[pred_id]

# 4) 예시 문장들로 테스트
clauses = [
    "제3조(보험금의 지급사유)\n회사는 피보험자에게 다음 사항 중 어느 하나의 사유가 발생한 경우에는 보험수익자에게 약정한 보험금((별표 1) 보험금 지급기준표 참조)을 지급한다.\n1. 연금개시 전 보험기간 중 장해분류표(별표 3 참조) 중 동일한 재해를 원인으로 여러 신 체부위의 장해지급률을 더하여 80퍼센트이상인 장해상태가 되었을 때(최초 1회한) 고도재 해장해보험금",
    "제2항 \n신탁계약에 의거 갑은 목적물에 대한 매도자의 지위를 가지는 자로서 신탁재산 및 신탁계약의 업무범위 내에서만 책임을 부담하며, 이 분양계약에 의하여 갑에게 발생하는 일체의 의무(해약금반환, 입주지연시 지체상금, 하자보수 등)는 신탁계약 위탁사인 정이 부담하기로 한다.\n갑  사업자(수탁자, 매도인)\n을  고객(매수인)\n정  제3자(위탁자)\n",
    "을(수탁자)은 본 계약 이행 중 발생한 손해에 대하여 회사(갑)의 과실이 없음을 증명하더라도, 발생 금액의 30%까지 책임을 부담한다.",
    "을은 본 계약과 관련하여 회사(갑)의 고의·중과실을 포함한 모든 귀책 사유에 대하여 어떠한 손해배상 청구도 하지 않으며, 발생 가능한 모든 손해를 스스로 부담한다.",
    "당사자 일방은 상대방에게 본 계약 해지 의사를 서면(전자문서 포함)으로 통지하고, 통지일로부터 30일이 경과하면 본 계약은 해지된다.",
    "회사는 피보험자에게 다음 중 어느 하나의 사유가 발생한 경우에는 보험수익자에게 약정한 보험금을 지급합니다.\n1. 보험기간 중에 상해의 직접결과로써 사망한 경우(질병으로 인한 사망은 제외합니다). ",
    "제2조(적용원칙) \n을이 주식의 위탁판매업무를 수행함에 있어서는 관련법령에 위배되지 않는 한 본 계약이 적용되며, 관련법령이 변경되거나 갑의 정관이 변경되는 경우에는 갑 과 을이 합의하여 본 계약을 변경할 수 있다."
]

for c in clauses:
    print(f"\"{c[:30]}...\" → {classify_clause(c)}")


"제3조(보험금의 지급사유)
회사는 피보험자에게 다음 사..." → 유리
"제2항 
신탁계약에 의거 갑은 목적물에 대한 매도자의 ..." → 불리
"을(수탁자)은 본 계약 이행 중 발생한 손해에 대하여 ..." → 불리
"을은 본 계약과 관련하여 회사(갑)의 고의·중과실을 포..." → 불리
"당사자 일방은 상대방에게 본 계약 해지 의사를 서면(전..." → 불리
"회사는 피보험자에게 다음 중 어느 하나의 사유가 발생한..." → 유리
"제2조(적용원칙) 
을이 주식의 위탁판매업무를 수행함에..." → 유리
